# Clustering and PCA

### 1. Librerías 
- Mushroom Dataset

- Enlace directo al dataset:

[Mushroom Dataset](https://www.kaggle.com/uciml/mushroom-classification)

Observaciones del profe: 
- Todas las variables de este dataset son categóricas
- Por lo tanto, exploraciones con scatterplot no nos serán útiles como en otros casos

- ¿Qué significa cada columna? 
- ¿Objetivo de este notebook?  La variable a predecir ``poisonous``, que es binaria.

- MIRAR LA METADA DE QUE SIGNIFICA CADA COLUMNA 


In [28]:
#%%capture 
# Hay que poner esta es una magi cell, en la primera linea de la celda, sin NADA antes, ni siquiera un comentario. Si pongo algo antes, Jupyter no lo reconoce como magic cell y falla. Sirve para ocultar la salida de esta celda, que es solo la instalación de librerías. No queremos que el output del notebook se llene de mensajes de instalación cada vez que lo ejecutamos.
# Y como no tengo un pip install debajo al que silenciar, no necesito esta cell magic. 

#%pip install missingno
#como estoy trabajando en local, con mi propio Python global, no necesito pip install dentro del notebook. Mis librerías ya están instaladas y persisten. Si estuviera trabajando en Google colab, que es un entorno en la nube donde cada vez abro una sesión, el entorno se reinicia desde cero - todas las librerías que no son estandar desaparecen. Por eso lo habitual es poner un pip install al principio.

In [29]:
# Carga de las librerías necesarias para la limpieza 

# - Para manipulación de datos -
import pandas as pd # Librería que utilizamos para trabajar con tablas de datos en Python (como excel pero en código). La usamos aqui para cargar el CSV, explorarlo, filtras columnas, contar nulos, crear nuevos DataFrame. 
import numpy as np #Librería de matemáticas y arrays numéricos. La usamos aquí para crear rangos de números (np.arrange), para trabajar con operaciones matemáticas durante PCA y KMeans, y para convertir DataFrames a arrays numéricos (np.array).

# - Para visualizacion - 
import seaborn as sns #Librería de visualización estadística. Hace gráficas bonitas con muy poco código. La usamos para los countplots del EDA, el catplot de clusters vs clase real, y el lineplot del método del codo. Alias sns. Viene de "Samuel Norman Seaborn", un personaje de The West Wing. Sí, en serio profe.
import matplotlib.pyplot as plt #Es la librería de base de gráficas en Python. La usamos para mostrar las gráficas de Seaborn, La usamos aquí para el scatterplot del PCA, controlar tamaños, títulos, y guardar gráficas. Seaborn hace las gráficas bonitas, matplotlib las personaliza y las muestra.

# - Para el modelado: clustering y reduccion de dimensionalidad - 
from sklearn.cluster import KMeans #Es el algoritmo de clustering KMeans de scikit-learn. Aqui lo usamos para agrupar los mushrooms en clusters in usar las etiquetas. El modelo buscará grupos naturales en los datos solo por similitud matemática. Punto clave a tener en cuenta: 'from sklearn.cluster' indica que viene del módulo de aprendizaje no supervisado de scikit-learn, y 'import KMeans' es el algoritmo específico que vamos a usar.
from sklearn.decomposition import PCA # Es la clase de PCA (Análisis de Componentes Principales) de Scikit-learn. Aqui la usamos para reducir dimensionalidad de los datos, es decir, para transformar las 22 columnas originales en solo 2 componentes principales que capturan la mayor parte de la variabilidad. Aqui la usamos para 2 usos distintos: 1. Visualización: comprimir 22 + variables en 2 dimensiones para poder pintarlas. y 2. Preprocesamiento: reducir dimensiones antes de entrenar modelos, para ver si mejoran o empeoran. Es útil para eliminar ruido y redundancia, y para entender mejor la estructura de los datos. PCA encuentra nuevas variables (componentes) que son combinaciones lineales de las originales, ordenadas por la cantidad de varianza que explican. La primera componente explica la mayor parte de la varianza, la segunda explica la siguiente mayor parte, y así sucesivamente. Al quedarnos solo con las primeras componentes, perdemos algo de información pero ganamos simplicidad y a menudo mejor rendimiento en modelos.

# - Para el modelado: clasificacion supervisada - 
from sklearn.ensemble import RandomForestClassifier # Es un clasificador supervisado que contruye muchos árboles de decisión y combina sus votos. Aqui lo usamos para comparar su desempeño con KMeans, pero esta vez usando las etiquetas reales de (comestible/venenoso) pra entrenar el modelo, y compararemos su precisión con lo que encuentra KMeans sin etiquetas. 'ensemble' significa que combina múltiples modelos - un "bosque" de árboles. Es un modelo poderoso que maneja bien datos con muchas características y es robusto al ruido. Al ser un modelo supervisado, necesita las etiquetas de clase para aprender, a diferencia de KMeans que es no supervisado.


# Para el preprocesamiento de datos antes de PCA y KMeans -
from sklearn.preprocessing import StandardScaler # Es una herramienta que escala los datos para que todas las variables tengan media 0 y desviación estandar 1. Es OBLIGATORIO antes de PCA porque PCA mide varianza, es decir, es sensible a la escala de variables. O sea, si una variable va de 0 a 1000 y otra de 0 a 1, PCA creerá que la primera es más importante solo por su escala. El escalado pone todas las variables en igualdad de condiciones. Regla de oro a recordar: 'StandardScaler' siempre antes de PCA y antes de KMeans. 
from sklearn.model_selection import train_test_split # Es una función que divide los datos en dos partes: entrenamiento y prueba. La usamos aqui para que el modelo aprenda con el 67% de los datos (train) y luego evaluamos su desenpeño con el 33% restante que nunca ha visto (test). Esto mide si el modelo generaliza bien o solo memorizó. (Me preguntaba porqué en este notebook se han configurado el 67/33 en lugar del típico 80/20 o 70/30, ya que es el punto donde suele haber suficientes datos para que el modelo aprenda bbien y suficiente para evaluarlo con fiabilidad. Se ve que utizaron este split porque el dataset tiene más de 8000 filas, así que incluso con 33% de test hay más de 2600 ejemplos paa evaluar, lo cual es robusto. Con datasets grandes podemos permitirnos dar mas datos al test sin perjudicar el entrenamiento. Regla general para decidir en el futuro: si mi dataset es pequeño (menos de 1000 filas), deberia usar 80/20 o incluso 90/10 para que el modelo tenga suficiente con qué aprender. Si mi dataset es más grande, puedo ser más generosa con el test())

# - Configuración de métricas de evaluación de modelos -
from sklearn.metrics import accuracy_score, classification_report # Son funciones para evaluar el desempeño de modelos de clasificación. Aquí las usamos para medir la precisión del Random Forest comparando sus predicciones con las etiquetas reales. Aqui el 'accuracy_score' nos da la proporción de predicciones correctas, mientras que el 'classification_report' nos da un desglose más detallado de precisión, luego, recall y F1-score para cada clase. Estas métricas nos ayudarán a entender no solo cuántas predicciones fueron correctas, sino también cómo se comporta el modelo con cada clase (comestible vs venenoso). Es decir, accuracy_score nos da el número rápido ("97% de acierto"). El classification_report nos dice si ese 97% real o si el modelo está fallando en una clase concreta (venenoso) y acertando todo en la otra (comestible), lo cual sería un problema. Por eso es importante mirar ambas métricas para tener una imagen completa del desempeño del modelo. 


### Además, añadiremos las constantes de configuración global:
 

 Añaidmos la semilla global RANDOM_STATE=42 para no tener que escribirla nuevamente en 5 sitios distintos. La definimos una vez y ya está. Si algún día decido cambiar la semilla, solo cambio este numero y afecta a todos los modelos.
 O sea, qué hace la semilla global esa? ella es el punto de partida de todos los algoritmos que tienen componente aleatorio (KMeans, RandomForest, train_test_split). 
 
 ¿Porqué? pues porque sin ella, cada vez que ejecute el notebook obetendré resultados ligeramente diferentes. 
 
 Con ella, los resultados son siempre iguales > reproducibilidad. Y por qué 42? Es una convención humorística de la comunidad Data Science (referencia a "The Hitchhiker's Guide to the Galaxy"). Cualquier número funcionaría. 
 
 Bueno, y por qué en capslock? porque las constantes globales siempre van en mayúscula en python (que no me acordaba). Esto le dice a cualquier lector "esto no debe cambiar dentro del código".

In [14]:

# Constante de Configuración de gráficas - La usamos para establecer el estilo visual de todas las gráficas del notebook de una vez. "whitegrid" = fondo blanco con cuadrícula. (10,6) = tamaño de gráficas en pulgadas.
sns.set_theme(style="whitegrid") 
plt.rcParams["figure.figsize"] = (10, 6)

# Semilla global — la usaremos en todos los modelos para reproducibilidad
RANDOM_STATE = 42 

print("✅ Librerías listas")


✅ Librerías listas


### 2. Leer conjunto de datos y primer vistazo

¿Qué haré? 

**Leer el CSV desde la carpeta data/raw y mostrar las primeras 5 filas** para confirmar que se cargó correctamente.

¿Por qué así? 

Usamos *rutas relativas* (../data/raw/mushrooms.csv) en lugar de rutas absolutaas (C:/Users/USUARIO/...) porque si en el futuro, llego a compartir el proyecto con alguien o lo subo a GitHub, la ruta absoluta rompería el ordenador de otra persona. La relativa funciona en cualquier máquina.

¿Qué alternativas tenemos?
 * pd.read_csv() con ruta absoluta > funciona pero no es portable
 * ucimlrepo > descarga el CSV automaticamente pero requiere internet
 * *Lo que elegí* > CSV local con ruta relariva > portable y profesional


In [32]:
# Cargar el dataset desde la carpeta de datos crudos (ruta relativa al notebook)
df = pd.read_csv("../data/raw/mushrooms.csv")

print("✅ Dataset cargado")

#Primer vistazo: shape nos dice (filas, columnas)
print(f"Dimensiones del dataset: {df.shape[0]} filas y {df.shape[1]} columnas")
print(f"Columnas: {df.columns.tolist()}")

# Las 5 primeras filas
df.head()

 

✅ Dataset cargado
Dimensiones del dataset: 8124 filas y 23 columnas
Columnas: ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


### 3. Exploración de datos (describe + dtypes)

¿Qué vamos hacer? Haremos dos inspecciones estándar que **siempre** se hacen al cargar un dataset:

1. .describe() - estadísticas básicas de cada columna
2. .info() o dtypes - tipo de dato de cada columna

¿Porqué? 
en un dataset numérico, *.describe()* te da medias, máximos, mínimos. Aquí todo es *object* (texto), así que lo que nos interesa confirmar es:

* ¿Todas las columnas son tipo *object*? (si alguna fuera int o float sería raro)
* ¿Cuántas filas tiene cada columna? (para detectar nulos indirectamente)



NOTA DEL PROFE

hay una variable que tiene un valor que demeos imputar 
knn imputer para imputar los valores.. 
ver en kaggle cual es la metadata 
MIRAR QUE HAY UNA COLUMNA / VALOR INTERROGANTE CON INTERROGACION - ELIMINAR? O USAR USAR UN ALGORITMO QUE SE LLAMA EL KNN IMPUTER - REEMPLAZAR EL VALOR, LOS VECINOS MAS CERCANOS, CUANTOS? TU LO DECIDES. ANTES DE LA CODIFICACION DE LAS VARIABLES CATEGORICAS. 
MATRIX DE CORRELACION . HAY UN MATRIX QUE SE LLAMA THE CRUMMER - CONSIDERARLA

https://analisisydecision.datanalytics.com/blog/grafico-de-correlaciones-entre-factores-grafico-de-la-v-de-cramer/

* 1. Descripción del conjunto de datos, estándard.

In [34]:
# Estadísticas descriptivas para variables categóricas


# 'include="all"' es necesario porque si no, describe() ignora columnas de texto
df.describe(include='all')


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


* 2. Información sobre el tipo de datos de cada feature.

Lo más importante a observar aqui es:

    -¿Todas las columnas son *object*?
    -¿ Hay alguna columna con menos de 8124 valores no-nulos? (señal de nulos) 

In [35]:
# Tipos de datos y memoria usada — confirma que todo es categórico (object)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

#### 4. Contar el número de nulos "reales" 

Igual que otras veces, una linea, contar los nulos por variable.

In [36]:
# Contar NaN reales (spoiler: no habrá ninguno en este dataset)
print("=== Nulos tipo NaN ===")
print(df.isnull().sum())

# Pero en este dataset los nulos están disfrazados como '?'
# Vamos a buscarlos contando cuántas veces aparece '?' en cada columna
print("\n=== Valores '?' por columna (nulos disfrazados) ===")
print((df == '?').sum()[( df == '?').sum() > 0])


=== Nulos tipo NaN ===
class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

=== Valores '?' por columna (nulos disfrazados) ===
stalk-root    2480
dtype: int64


    *stalk-root tiene 2480 valores ? - eso es el 30.5% de laas filas del dataset. No es poca cosa. Luego tendremos que decidir qué hacer con ellos.*

* *Lección clave para recordar en futuros proyectos:
En proyectos reales, los nulos rara vez vienen como NaN. Pueden ser ?, N/A, none, -1, 0, o incluso un espacio en blanco. Siempre hay que buscarlos de forma activa, no solo con .isnull().*

## 5. Buscar valores extraños. Para ello, ver los valores únicos por feature

¿Qué vamos a hacer?

El profe pide un DataFrame de dos columnas: la feature y cuántos valores únicos tiene. Esto sirve para dos cosas:

1. Detectar columnas con un solo valor > esas no aportan información, considerar si se eliminan.

2. Ver la riqueza de cada variable categórica

In [37]:
# Obtener un nuevo dataframe de dos columnas donde en la primera estén las features (features) y en la otra los valores únicos
# Crear DataFrame con número de valores únicos por columna


# nunique() cuenta cuántos valores distintos hay en cada columna
# asociados (n_values). Luego ordenamos por número de valores únicos para ver cuáles son las columnas con menos variedad (posibles candidatas a eliminar o a convertir en binarias).
unique_values = pd.DataFrame({
    'feature': df.columns,
    'n_values': [df[col].nunique() for col in df.columns]
}).sort_values('n_values')

print(unique_values.to_string(index=False))


                 feature  n_values
               veil-type         1
                   class         2
         gill-attachment         2
            gill-spacing         2
             stalk-shape         2
               gill-size         2
                 bruises         2
             ring-number         3
stalk-surface-above-ring         4
              veil-color         4
             cap-surface         4
stalk-surface-below-ring         4
               ring-type         5
              stalk-root         5
               cap-shape         6
              population         6
                 habitat         7
                    odor         9
  stalk-color-above-ring         9
  stalk-color-below-ring         9
       spore-print-color         9
               cap-color        10
              gill-color        12


#### Tratar aquellos valores que entendamos que sean nulos


In [ ]:
# Imputaciones. Podéis quitar esos puntos (fila entera), imputar con la moda o dejar ese valor como una posibilidad más.

#### Mirad cuántos valores hay en cada feature, ¿Todas las features aportan información? Si alguna no aporta información, eliminadla

In [ ]:
# Dejar por el camino si procede.

#### Separar entre variables predictoras y variables a predecir

In [ ]:
# La variable que trata de predecir este conjunto de datos es 'poisonous'.
y =
X =

SyntaxError: invalid syntax (<ipython-input-9-411deef15e8a>, line 2)

#### Codificar correctamente las variables categóricas a numéricas

In [ ]:
# One Hot Encoder (una linea).

#### Train test split

In [ ]:
# Os lo dejamos a todos igual
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## PCA

Es un conjunto de datos del que aún no hemos visto nada (no tenemos graficas) así que vamos a hacer algunas. Tenemos el problema de que son muchas variables, **PCA al rescate**: le pedimos que nos de dos dimensiones y las pintamos, sabemos que serán **aquellas que retengan más información**.

In [ ]:
pca =       # metodo de sklearn
pca.fit(X_train)

# Representar en un scatterplot y poner en color las etiquetas de entrenamiento

Parece que está bastante separadito, parece que a ojo mucho se puede ver :)

Igualmente, vamos a entrenar un clasificador a ver qué tal lo hace antes de editar más

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1. Definir el clasificador y el número de estimadores
# 2. Entrenar en train
# 3. Calcular la precisión sobre test

Es un conjunto sencillo y Random Forest es muy bueno en su trabajo, Igualmente, vamos a ver qué tamaño tenemos de dataset:


In [ ]:
X_train.shape

¿Muchas features no? Vamos a reducir las usando PCA.

In [ ]:
n_features = # definir un rango de valores a probar
scores = []

for n in n_features:

    # Hacer PCA sobre X_train
    # 1. Definir PCA
    # 2. Aprender PCA sobre X_train

    # Entrenar Random Forest
    # 1. Definir el RF
    # 2. Entrenar clasificador

    # Guardar el score


sns.lineplot(x=n_features, y=scores)


Vale, estamos viendo que a partir de unas 10 features ya tenemos el score que queríamos y además hemos reducido las variables a un 10% de las que teníamos, incluso menos que las variables originales.

## Clustering

Viendo que el conjunto de datos es sencillito, podemos intentar hacer algo de clustering a ver qué información podemos obtener.

El primer paso va a ser importar la función de Kmeans de sklearn, y a partir de ahi, vamos a buscar el valor óptimo de clusters. Como hemos visto anteriormente, este valor lo obtenemos, por ejemplo, del codo de la gráfica que representa el total de las distancias de los puntos a los centros de los clusters asociados. Os dejo la página de la documentación de sklearn para que lo busquéis:

[K-Means on sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

Con esto solo hay que ahora generar los modelos de kmeans, evaluar y pintar la gráfica para los valores de ``k`` que establezcais.




In [ ]:
from sklearn.cluster import KMeans

scores = []
k_values = # definir un rango
for a in k_values:

    # Definir Kmeans y ajustar
    # Guardar la predicción

sns.lineplot(x=k_values, y=scores)

Con el valor que hayáis obtenido de la gráfica, podéis obtener una buena aproximación de Kmeans y con ello podemos pasar a explorar cómo de bien han separado la información los distintos clusters. Para ello, se va a hacer un ``catplot``, seaborn os lo hará solito. Con esto lo que se pretende ver es la distribución de la varaible a predecir en función del cluster que haya determinado Kmeans.

In [ ]:
# Aprender Kmeans con el valor de K obtenido.

kmeans = # Definir y entrenar Kmeans.

# Preparar el catplot.


# Pintar.
ax = sns.catplot(col=, x=, data=, kind='count',col_wrap=4)

Vamos a ver qué tal queda esto pintado. Para ello, repetimos el scatterplot de antes pero usando como color el cluster asignado por kmeans.

In [ ]:
# Entrenar PCA para representar.

# Usar un color por cada cluster.


¿Es bastante parecido no? No es tan bueno como el Random Forest, pero ha conseguido identificar bastante bien los distintos puntos del dataset sin utilizar las etiquetas. De hecho, el diagrama de factor que hemos visto antes muestra que solo un par de clusters son imprecisos. Si no hubieramos tenido etiquetas esta aproximacion nos hubiera ayudado mucho a clasificar los distintos tipos de hongos.